In [2]:
import pandas as pd
import pyarrow as pa
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import calendar

import geopandas
from shapely.geometry import Point

In [3]:
base_dir_path = "Data/Taxi data"

In [4]:
# taxi_data_sample = pd.read_parquet("Data/Taxi Data/2013/yellow_tripdata_2013-01.parquet")

In [5]:
# taxi_data_sample2 = pd.read_parquet("Data/Taxi Data/2023/yellow_tripdata_2023-12.parquet")

In [6]:
# taxi_data_sample.info()

In [7]:
# taxi_data_sample2.info()

In [8]:
def import_and_combine_nyctaxi_data(base_dir, year):
    year_dir = os.path.join(base_dir, f"{year}")
    all_dataframes = [] 

#     for month in range(1, 13):
#         month_dir = os.path.join(year_dir, f"{month}_{calendar.month_name[month]}")
#         if os.path.exists(month_dir):  # Check if the month directory exists
    for filename in os.listdir(year_dir):
        if filename.endswith('.parquet'):  
            file_path = os.path.join(year_dir, filename)
            data = pd.read_parquet(file_path)
            keep_columns = ['tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count','PULocationID','DOLocationID']
            df = pd.DataFrame(data=data, columns = keep_columns)
            df.dropna(inplace=True)
            X = ['passenger_count','PULocationID','DOLocationID']
            for x in X:
                df[x] = df[x].astype(int)
            all_dataframes.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    return combined_df


base_directory = base_dir_path
year_to_import = 2021
combined_data = import_and_combine_nyctaxi_data(base_directory, year_to_import)
# combined_data.head(30)
combined_data.shape

(23839125, 5)

In [9]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23839125 entries, 0 to 23839124
Data columns (total 5 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[ns]
 1   tpep_dropoff_datetime  datetime64[ns]
 2   passenger_count        int64         
 3   PULocationID           int64         
 4   DOLocationID           int64         
dtypes: datetime64[ns](2), int64(3)
memory usage: 909.4 MB


In [10]:
df_temp = combined_data.sort_values(by='tpep_pickup_datetime')
df_temp

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID
15042073,2002-12-31 23:06:55,2002-12-31 23:08:03,1,264,264
16537342,2003-01-01 00:07:17,2003-01-01 14:16:59,1,193,193
3323824,2003-03-26 21:07:28,2003-03-27 13:30:46,1,193,193
8956244,2008-12-31 22:20:23,2008-12-31 22:55:19,1,238,234
1442027,2008-12-31 23:02:19,2009-01-01 18:14:43,6,233,229
...,...,...,...,...,...
3981003,2021-04-28 18:28:11,2021-04-28 18:40:24,1,140,234
3981004,2021-04-28 18:43:30,2021-04-29 02:02:52,1,170,87
3981005,2021-05-20 02:07:54,2021-05-20 02:13:51,1,261,13
3985543,2021-05-20 02:40:50,2021-05-20 02:47:10,1,13,88


In [11]:
combined_data = combined_data[combined_data['tpep_pickup_datetime'].dt.year==2021]
combined_data = combined_data[combined_data['tpep_dropoff_datetime'].dt.year==2021]

In [12]:
# combined_data.isnull().sum()

In [13]:
# temp_data = pd.read_csv("Data/Citib ike Data/2021-citibike-tripdata/1_January/202101-citibike-tripdata_1.csv")
# temp_data.info()

In [14]:
# temp_data2 = pd.read_csv("Data/Citibike Data/2021-citibike-tripdata/12_December/202112-citibike-tripdata_1.csv")
# temp_data2.info()

In [15]:
combined_data

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID
0,2020-03-01 00:31:13,2020-03-01 01:01:42,1,88,255
1,2020-03-01 00:08:22,2020-03-01 00:08:49,1,193,193
2,2020-03-01 00:52:18,2020-03-01 00:59:16,1,246,90
3,2020-03-01 00:47:53,2020-03-01 00:50:57,2,151,238
4,2020-03-01 00:43:19,2020-03-01 00:58:27,0,79,261
...,...,...,...,...,...
23839120,2020-06-30 23:07:00,2020-06-30 23:08:46,1,74,194
23839121,2020-06-30 23:24:49,2020-06-30 23:32:00,1,138,70
23839122,2020-06-30 23:57:38,2020-07-01 00:24:57,1,138,259
23839123,2020-06-30 23:08:03,2020-06-30 23:26:37,1,41,41


In [16]:
data = combined_data

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23838665 entries, 0 to 23839124
Data columns (total 5 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[ns]
 1   tpep_dropoff_datetime  datetime64[ns]
 2   passenger_count        int64         
 3   PULocationID           int64         
 4   DOLocationID           int64         
dtypes: datetime64[ns](2), int64(3)
memory usage: 1.1 GB


In [18]:
def preprocessing(data):
    
#     keep_columns = ['tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count','PULocationID','DOLocationID']
#     df_preprocess = pd.DataFrame(data=data, columns = keep_columns)
    df_preprocess = pd.DataFrame(data=data)
    df_preprocess.dropna(inplace=True)

#     X = ['passenger_count','PULocationID','DOLocationID']
#     for x in X:
#         df_preprocess[x] = df_preprocess[x].astype(int)

    df_preprocess['tpep_pickup_datetime'] = pd.to_datetime(df_preprocess['tpep_pickup_datetime'])
    df_preprocess['unix_start_time'] = df_preprocess['tpep_pickup_datetime'].astype('int64') // 10**9

    df_preprocess['tpep_dropoff_datetime'] = pd.to_datetime(df_preprocess['tpep_dropoff_datetime'])
    df_preprocess['unix_stop_time'] = df_preprocess['tpep_dropoff_datetime'].astype('int64') // 10**9  
    
    df_preprocess.drop(columns = ['tpep_pickup_datetime','tpep_dropoff_datetime'], inplace=True)

    df_preprocess['diff'] = df_preprocess['unix_stop_time'] - df_preprocess['unix_start_time']
    df_preprocess = df_preprocess[df_preprocess['diff'] <= 3600]

    df_preprocess = df_preprocess[df_preprocess['PULocationID']!=df_preprocess['DOLocationID']]
    df_preprocess = df_preprocess[df_preprocess['unix_start_time']!=df_preprocess['unix_stop_time']]
    df_preprocess = df_preprocess[
        (df_preprocess['PULocationID'] != 264) & (df_preprocess['PULocationID'] != 265) & (df_preprocess['DOLocationID'] != 264) & (df_preprocess['DOLocationID'] != 265)
    ]

    df_processed = df_preprocess.sort_values(by='unix_start_time')
    df_processed.reset_index(inplace=True)

    final_processed = df_processed[['unix_start_time','unix_stop_time','passenger_count','PULocationID','DOLocationID']]
#     for col in final_processed.columns:
#         final_processed[col] = final_processed[col].astype(int)
    return final_processed

In [19]:
export_df = preprocessing(data)
export_df

,unix_start_time,unix_stop_time,passenger_count,PULocationID,DOLocationID
0,1577836800,1577837583,1,68,170
1,1577836802,1577837809,1,68,246
2,1577836803,1577837584,2,79,162
3,1577836808,1577837156,1,236,262
4,1577836809,1577837237,2,90,170
...,...,...,...,...,...
22042900,1609458849,1609459022,6,262,75
22042901,1609458851,1609459072,1,125,234
22042902,1609458897,1609459059,1,238,239
22042903,1609458922,1609459113,1,68,186


In [20]:
export_table = pa.Table.from_pandas(export_df)

In [21]:
# export_df.isnull().sum()
# export_df.to_csv("ProcessedTaxiData/TaxiProcessed2023",index=False)

pa.parquet.write_table(export_table, "ProcessedTaxiData/TaxiProcessed2021")